# UAV arm #

This script defines robot
1. DH parameter (**ongoing, check diagram, axes, dh parameters**)
2. Transformation Matrix (**ongoing**)
3. Forward kinematics (**ongoing**)
4. Jacobian (**ongoing**)
4. Singularity analysis (**ongoing**)

In [179]:
import numpy as np
import sympy
sympy.init_printing(use_unicode=True)

# Define a robot parameters

In [180]:
# apply link lengths
l_3 = 1
l_4 = 1
l_5 = 1
l_6 = 1

## 1. DH parameters

DH parameters are obtained using modified DH convention (*ith joint defined by ith coordinate*)

**NOTE:**
1. Check axes selection
2. To change the body diagram image, change svg file in images folder. I used inkspace to draw
3. For modified DH_convention (and it's difference from classic one), refer to ***DH_convention.pdf***
4. Solve later: convert the T0_6 to  the quaternion representation. In the end, the equation should be: *x, y, z, q1, q2, q3, q4*

![title](images/for_kin_body_diagram_3.svg)

In [181]:
# dh parameters
theta_2, theta_3, theta_4, theta_5, theta_6 = sympy.symbols(['theta_2', 'theta_3', 'theta_4', 'theta_5', 'theta_6'])
d_1 = sympy.Symbol('d_1')


thetas = [0, theta_2, theta_3, theta_4, theta_5, theta_6]
d = [d_1, 0, l_3, 0, 0, l_6]
r = [0, 0, 0, l_4, l_5, 0]
alphas = [-sympy.pi/2, -sympy.pi/2, -sympy.pi/2, 0, 0, sympy.pi/2]

Confirm table values

In [182]:
thetas

In [183]:
d

In [184]:
r

In [185]:
alphas

In [186]:
# just check as a table
for i in range(6):
    print(i+1, '    ',  thetas[i],
          '    ',  d[i],
         '    ',  r[i],
         '    ',  alphas[i])

1      0      d_1      0      -pi/2
2      theta_2      0      0      -pi/2
3      theta_3      1      0      -pi/2
4      theta_4      0      1      0
5      theta_5      0      1      0
6      theta_6      1      0      pi/2


## 2.Transformation matrix

Quick look: https://en.wikipedia.org/wiki/Denavit%E2%80%93Hartenberg_parameters#Modified_DH_parameters

Otherwise the equation are given in the pdf also

In [187]:
# these variable must be changed accordingly
theta_curr, alpha_pre, r_pre, d_curr = sympy.symbols(['theta_i', r'\alpha_{i-1}', 'r_{i-1}', 'd_i'])

In [188]:
T_expr = sympy.Matrix([ [sympy.cos(theta_curr), -sympy.sin(theta_curr), 0, r_pre],
                        [sympy.sin(theta_curr)*sympy.cos(alpha_pre), sympy.cos(theta_curr)*sympy.cos(alpha_pre), -sympy.sin(alpha_pre), -d_curr*sympy.sin(alpha_pre)],
                      [sympy.sin(theta_curr)*sympy.sin(alpha_pre), sympy.cos(theta_curr)*sympy.sin(alpha_pre), sympy.cos(alpha_pre), -d_curr*sympy.cos(alpha_pre)],
                      [0, 0, 0, 1]])

In [189]:
T_expr

⎡         cos(θᵢ)                   -sin(θᵢ)                   0              
⎢                                                                             
⎢sin(θᵢ)⋅cos(\alpha_{i-1})  cos(\alpha_{i-1})⋅cos(θᵢ)  -sin(\alpha_{i-1})  -dᵢ
⎢                                                                             
⎢sin(\alpha_{i-1})⋅sin(θᵢ)  sin(\alpha_{i-1})⋅cos(θᵢ)  cos(\alpha_{i-1})   -dᵢ
⎢                                                                             
⎣            0                          0                      0              

    r_{i-1}       ⎤
                  ⎥
⋅sin(\alpha_{i-1})⎥
                  ⎥
⋅cos(\alpha_{i-1})⎥
                  ⎥
       1          ⎦

In [190]:
# initialize transformation variables
T0_1, T1_2, T2_3, T3_4, T4_5, T5_6, T0_6 = sympy.symbols(['T^0_1', 'T^1_2', 'T^2_3', 'T^3_4', 'T^4_5', 'T^5_6', 'T^0_6'])

In [191]:
# 0 ---> 1
_0_1 = {
    theta_curr:thetas[0],
    alpha_pre:alphas[0],
    r_pre:r[0],
    d_curr:d[0]
}
T0_1 = T_expr.subs(_0_1).evalf()

In [192]:
T0_1

⎡1.0   0     0    0 ⎤
⎢                   ⎥
⎢ 0    0    1.0  d₁ ⎥
⎢                   ⎥
⎢ 0   -1.0   0    0 ⎥
⎢                   ⎥
⎣ 0    0     0   1.0⎦

In [193]:
# 1 ---> 2
_1_2 = {
    theta_curr:thetas[1],
    alpha_pre:alphas[1],
    r_pre:r[1],
    d_curr:d[1]
}
T1_2 = T_expr.subs(_1_2).evalf()

In [194]:
T1_2

⎡cos(θ₂)   -sin(θ₂)   0    0 ⎤
⎢                            ⎥
⎢   0         0      1.0   0 ⎥
⎢                            ⎥
⎢-sin(θ₂)  -cos(θ₂)   0    0 ⎥
⎢                            ⎥
⎣   0         0       0   1.0⎦

In [195]:
# 2 ---> 3
_2_3 = {
    theta_curr:thetas[2],
    alpha_pre:alphas[2],
    r_pre:r[2],
    d_curr:d[2]
}
T2_3 = T_expr.subs(_2_3).evalf()

In [196]:
T2_3

⎡cos(θ₃)   -sin(θ₃)   0    0 ⎤
⎢                            ⎥
⎢   0         0      1.0  1.0⎥
⎢                            ⎥
⎢-sin(θ₃)  -cos(θ₃)   0    0 ⎥
⎢                            ⎥
⎣   0         0       0   1.0⎦

In [197]:
# 3 ---> 4
_3_4 = {
    theta_curr:thetas[3],
    alpha_pre:alphas[3],
    r_pre:r[3],
    d_curr:d[3]
}
T3_4 = T_expr.subs(_3_4).evalf()

In [198]:
T3_4

⎡cos(θ₄)  -sin(θ₄)   0   1.0⎤
⎢                           ⎥
⎢sin(θ₄)  cos(θ₄)    0    0 ⎥
⎢                           ⎥
⎢   0        0      1.0   0 ⎥
⎢                           ⎥
⎣   0        0       0   1.0⎦

In [199]:
# 4 ---> 5
_4_5 = {
    theta_curr:thetas[4],
    alpha_pre:alphas[4],
    r_pre:r[4],
    d_curr:d[4]
}
T4_5 = T_expr.subs(_4_5).evalf()

In [200]:
T4_5

⎡cos(θ₅)  -sin(θ₅)   0   1.0⎤
⎢                           ⎥
⎢sin(θ₅)  cos(θ₅)    0    0 ⎥
⎢                           ⎥
⎢   0        0      1.0   0 ⎥
⎢                           ⎥
⎣   0        0       0   1.0⎦

In [201]:
# 5 ---> 6
_5_6 = {
    theta_curr:thetas[5],
    alpha_pre:alphas[5],
    r_pre:r[5],
    d_curr:d[5]
}
T5_6 = T_expr.subs(_5_6).evalf()

In [202]:
T5_6

⎡cos(θ₆)  -sin(θ₆)   0     0  ⎤
⎢                             ⎥
⎢   0        0      -1.0  -1.0⎥
⎢                             ⎥
⎢sin(θ₆)  cos(θ₆)    0     0  ⎥
⎢                             ⎥
⎣   0        0       0    1.0 ⎦

Equation T0_6 results to 9 equations, however if we can manage to represent in terms of quaternions it's going to be 9 equations (x, y, z, q1, q2, q3, q4)
Hence,
1. Does visualization tool work with quaternions?
2. Can we convert it easily? Any books/search?

If above answers are yes, then we need to conver it to quaternion, **also quaternion doesn't have a problem of gimbal lock:** https://en.wikipedia.org/wiki/Gimbal_lock

# 3. Forward Kinematics

The forward kinematics is obtained as following:

\begin{align}
T^6_0 = [R_{3x3}; p_{3x1}]
\end{align}

where $R_{3x3}$ is an orientation and $p_{3x1}$ is end-effector cartesian position respect to base

In [203]:
# complete base to end-effector transformation
T0_6 = T0_1*T1_2*T2_3*T3_4*T4_5*T5_6
T0_6 = sympy.simplify(T0_6)

In [204]:
T0_6

⎡-1.0⋅sin(θ₂)⋅sin(θ₆) + 1.0⋅cos(θ₂)⋅cos(θ₆)⋅cos(θ₃ + θ₄ + θ₅)  -1.0⋅sin(θ₂)⋅co
⎢                                                                             
⎢-1.0⋅sin(θ₂)⋅cos(θ₆)⋅cos(θ₃ + θ₄ + θ₅) - 1.0⋅sin(θ₆)⋅cos(θ₂)  1.0⋅sin(θ₂)⋅sin
⎢                                                                             
⎢               1.0⋅sin(θ₃ + θ₄ + θ₅)⋅cos(θ₆)                                 
⎢                                                                             
⎣                             0                                               

s(θ₆) - 1.0⋅sin(θ₆)⋅cos(θ₂)⋅cos(θ₃ + θ₄ + θ₅)  1.0⋅sin(θ₃ + θ₄ + θ₅)⋅cos(θ₂)  
                                                                              
(θ₆)⋅cos(θ₃ + θ₄ + θ₅) - 1.0⋅cos(θ₂)⋅cos(θ₆)   -1.0⋅sin(θ₂)⋅sin(θ₃ + θ₄ + θ₅) 
                                                                              
-1.0⋅sin(θ₆)⋅sin(θ₃ + θ₄ + θ₅)                     -1.0⋅cos(θ₃ + θ₄ + θ₅)     
                                                   

# 4. Jacobian

The following code calculates geometrical Jacobian

In [205]:
# select rotations
R0_1 = T0_1[0:3,0:3]
R1_2 = T1_2[0:3,0:3]   
R2_3 = T2_3[0:3,0:3]  
R3_4 = T3_4[0:3,0:3]   
R4_5 = T4_5[0:3,0:3] 
R5_6 = T5_6[0:3,0:3]

# end effector position
p6 = T0_6[0:3,3]

In [206]:
p6

⎡    -1.0⋅sin(θ₂) + 1.0⋅sin(θ₃ + θ₄ + θ₅)⋅cos(θ₂) + 1.0⋅cos(θ₂)⋅cos(θ₃) + 1.0⋅
⎢                                                                             
⎢1.0⋅d₁ - 1.0⋅sin(θ₂)⋅sin(θ₃ + θ₄ + θ₅) - 1.0⋅sin(θ₂)⋅cos(θ₃) - 1.0⋅sin(θ₂)⋅co
⎢                                                                             
⎣                       1.0⋅sin(θ₃) + 1.0⋅sin(θ₃ + θ₄) - 1.0⋅cos(θ₃ + θ₄ + θ₅)

cos(θ₂)⋅cos(θ₃ + θ₄)    ⎤
                        ⎥
s(θ₃ + θ₄) - 1.0⋅cos(θ₂)⎥
                        ⎥
                        ⎦

In [207]:
# direction of z(i-1) axis
k = sympy.Matrix([0, 0, 1])
z0 = k;
z1 = R0_1*k;
z2 = R0_1*R1_2*k;
z3 = R0_1*R1_2*R2_3*k;
z4 = R0_1*R1_2*R2_3*R3_4*k;
z5 = R0_1*R1_2*R2_3*R3_4*R4_5*k;

In [211]:
# define Jacobian
J = sympy.zeros(6,6)
J[0:3,0] = sympy.diff(p6,d_1)
J[0:3,1] = sympy.diff(p6,theta_2)
J[0:3,2] = sympy.diff(p6,theta_3)
J[0:3,3] = sympy.diff(p6,theta_4)
J[0:3,4] = sympy.diff(p6,theta_5)
J[0:3,5] = sympy.diff(p6,theta_6)

J[3:6,0] = sympy.diff(z0)
J[3:6,1] = sympy.diff(z1)
J[3:6,2] = sympy.diff(z2)
J[3:6,3] = sympy.diff(z3)
J[3:6,4] = sympy.diff(z4)
J[3:6,5] = sympy.diff(z5)

J = sympy.simplify(J)

In [212]:
J

⎡ 0   -1.0⋅sin(θ₂)⋅sin(θ₃ + θ₄ + θ₅) - 1.0⋅sin(θ₂)⋅cos(θ₃) - 1.0⋅sin(θ₂)⋅cos(θ
⎢                                                                             
⎢1.0  1.0⋅sin(θ₂) - 1.0⋅sin(θ₃ + θ₄ + θ₅)⋅cos(θ₂) - 1.0⋅cos(θ₂)⋅cos(θ₃) - 1.0⋅
⎢                                                                             
⎢ 0                                                 0                         
⎢                                                                             
⎢ 0                                                 0                         
⎢                                                                             
⎢ 0                                                 0                         
⎢                                                                             
⎣ 0                                                 0                         

₃ + θ₄) - 1.0⋅cos(θ₂)  1.0⋅(-sin(θ₃) - sin(θ₃ + θ₄) + cos(θ₃ + θ₄ + θ₅))⋅cos(θ
                                                   

**SOMEHOW ABOVE JACOBIAN SAYS THAT THE ROBOT IS IN ALWAYS SINGULAR POSE???**